Importando Bibliotecas

In [ ]:
import numpy as np
import random as rd

Inicializando a População

In [ ]:
num_individuos = 1000
num_rainhas = 8
populacao_inicial = np.zeros((num_individuos, num_rainhas), dtype=int)

for i in range(num_individuos):
  for j in range(num_rainhas):
    populacao_inicial[i][j] = rd.randrange(4)

#for i in range(num_individuos):
#  print(populacao_inicial[i])

Definindo a função de fitness

In [ ]:
def fitness(ind):
  
  f = 0 #fitness

  for i in range(num_rainhas-1):
    for j in range(i+1, num_rainhas):
      if ind[i] != ind[j] and abs(i-j) != abs(ind[i]-ind[j]):
        f += 1

  return f

def retorna_fitness(populacao):
  
  fit = []
  
  for i in range(num_individuos):
    f = fitness(populacao[i])
    fit.append(f)

  return fit
  

In [ ]:
#for i in range(10):
  #print(fitness(populacao_inicial[i]))

Definindo função de seleção por torneio

In [ ]:
def selecao_torneio(fitnesses):
  
  ind1 = -1
  ind2 = -1

  while ind1 == ind2:
    '''
    # Torneio 1
    sorteados = rd.sample(range(0, num_individuos), 2)
    if fitnesses[sorteados[0]] > fitnesses[sorteados[1]]:
      ind1 = sorteados[0]
    else:
      ind1 = sorteados[1]

    # Torneio 2
    sorteados = rd.sample(range(0, num_individuos), 2)
    if fitnesses[sorteados[0]] > fitnesses[sorteados[1]]:
      ind2 = sorteados[0]
    else:
      ind2 = sorteados[1]
    '''
    sorteados = rd.sample(range(0, num_individuos), 3)
    if fitnesses[sorteados[0]] > fitnesses[sorteados[1]] and fitnesses[sorteados[0]] > fitnesses[sorteados[2]]:
      ind1 = sorteados[0]
    elif fitnesses[sorteados[1]] > fitnesses[sorteados[2]]:
      ind1 = sorteados[1]
    else:
      ind1 = sorteados[2]

    sorteados = rd.sample(range(0, num_individuos), 3)
    if fitnesses[sorteados[0]] > fitnesses[sorteados[1]] and fitnesses[sorteados[0]] > fitnesses[sorteados[2]]:
      ind2 = sorteados[0]
    elif fitnesses[sorteados[1]] > fitnesses[sorteados[2]]:
      ind2 = sorteados[1]
    else:
      ind2 = sorteados[2]
    

  return ind1,ind2

  '''
  ind1 = max(fitnesses[sorteados[0]], fitnesses[sorteados[1]], fitnesses[sorteados[2]])
  '''

In [ ]:
def selecao_torneio(fitnesses):
  
  ind1 = -1
  ind2 = -1

  while ind1 == ind2:
    # Torneio 1
    sorteados = rd.sample(range(0, num_individuos), 3)
    ind1 = max(fitnesses[sorteados[0]], fitnesses[sorteados[1]], fitnesses[sorteados[2]])

    # Torneio 2
    sorteados = rd.sample(range(0, num_individuos), 3)
    ind2 = max(fitnesses[sorteados[0]], fitnesses[sorteados[1]], fitnesses[sorteados[2]])

  return ind1,ind2

Definindo função de cruzamento de 1 ponto

In [ ]:
def cruzamento(ids, populacao):
  
  ponto = rd.randrange(1, num_rainhas)

  pai1 = populacao[ids[0]]
  pai2 = populacao[ids[1]]

  filho1 = np.concatenate([pai1[:ponto],pai2[ponto:]])
  filho2 = np.concatenate([pai2[:ponto],pai1[ponto:]])

  return filho1,filho2


Definindo a função de elitismo

In [ ]:
def elitismo(fitnesses):

  id1 = fitnesses.index(max(fitnesses))

  fitnesses.pop(id1)
  id2 = fitnesses.index(max(fitnesses))

  return id1,id2

Definindo a função de mutação

In [ ]:
def mutacao(filhos, taxa):

  for i in range(len(filhos)):
    if rd.random() < taxa:
      pos = rd.randrange(num_rainhas)
      filhos[i][pos] = rd.randrange(8)

  return filhos


Código principal

In [ ]:
# Número de gerações
num_geracoes = 100
fitness_otimo = 28
repeticoes = 0
fit_anterior = 0

for it in range(num_geracoes):

  # Nova populacao vazia
  nova_populacao = np.zeros((num_individuos, num_rainhas), dtype=int)

  # Calcula o fitness da população
  fit = retorna_fitness(populacao_inicial)

  # Imprime o melhor individuo
  id_melhor = fit.index(max(fit))
  print(f"M{it}:", '  '.join(map(str,populacao_inicial[id_melhor])), "Fit =", fit[id_melhor])
  print()

  # Contagem de repetições do maior fitness
  if(fit_anterior == fit[id_melhor]):
    repeticoes += 1
  else:
    repeticoes = 0

  fit_anterior = fit[id_melhor]

  # Condições de parada
  if fit[id_melhor] == fitness_otimo or repeticoes == 10:
    break

  # Elitismo
  elite = elitismo(fit.copy()) # Copiando por causa da referencia do vetor
  nova_populacao[0] = populacao_inicial[elite[0]]
  nova_populacao[1] = populacao_inicial[elite[1]]

  # Gera os filhos restantes para completar a população
  num_filhos = 2
  while num_filhos < num_individuos:

    # Seleção por torneio
    ind_vencedores = selecao_torneio(fit)
    
    # Cruzamento
    filhos = cruzamento(ind_vencedores, populacao_inicial)

    # Mutacao (50% de chance de mutação em cada indivíduo)
    filhos = mutacao(filhos, 0.5)

    # Coloca os filhos na nova população
    nova_populacao[num_filhos] = filhos[0]
    nova_populacao[num_filhos+1] = filhos[1]

    # Aumenta o número de filhos
    num_filhos = num_filhos + 2

  # Substitui a populacao antiga pela atual
  populacao_inicial = nova_populacao.copy()


M0: 2  2  1  3  0  0  3  1 Fit = 23

M1: 2  2  1  3  0  0  3  1 Fit = 23

M2: 0  0  2  4  1  7  3  1 Fit = 24

M3: 0  0  2  4  1  7  3  1 Fit = 24

M4: 0  7  2  4  7  0  3  5 Fit = 25

M5: 0  7  2  4  7  0  3  5 Fit = 25

M6: 3  0  2  4  1  7  4  1 Fit = 26

M7: 3  0  2  4  1  7  4  1 Fit = 26

M8: 3  0  2  4  1  7  4  1 Fit = 26

M9: 3  0  2  4  1  7  4  1 Fit = 26

M10: 3  0  2  4  1  7  4  1 Fit = 26

M11: 3  0  2  4  1  7  4  1 Fit = 26

M12: 3  0  2  4  1  7  4  1 Fit = 26

M13: 3  0  2  4  1  7  4  1 Fit = 26

M14: 3  0  2  4  1  7  4  1 Fit = 26

M15: 3  0  2  4  1  7  4  1 Fit = 26

M16: 3  0  2  4  1  7  4  1 Fit = 26

